In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

In [3]:
from counterfactuals.datasets import HelocDataset
from counterfactuals.discriminative_models import MultilayerPerceptron

In [4]:
dataset = HelocDataset("../data/heloc.csv")

In [5]:
disc_model = MultilayerPerceptron(dataset.X_test.shape[1], [512, 512], 2)
disc_model.load("heloc_disc_model.pt")

/Users/lukasz/Genwro/counterfactuals/counterfactuals/discriminative_models/multilayer_perceptron.py:109: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(t

In [6]:
class DiscModelWrapper:
    def __init__(self, model):
        self.model = model
        self.model_type = "classifier"

    def get_output(
        self, input_instance, model_score=True, transform_data=False, out_tensor=False
    ):
        """returns prediction probabilities

        :param input_tensor: test input.
        :param transform_data: boolean to indicate if data transformation is required.
        """
        input_tensor = input_instance
        if not torch.is_tensor(input_instance):
            input_tensor = torch.tensor(input_instance.to_numpy()).float()
        out = self.model(input_tensor).float()
        if not out_tensor:
            out = out.data.numpy()
        if model_score is False:
            out = np.round(out)  # TODO need to generalize for n-class classifier
        return out

In [7]:
from counterfactuals.cf_methods.glance import Glance

model = DiscModelWrapper(disc_model)

global_glance = Glance(dataset, model)
global_glance.prep(method_to_use="dice")

merge_history = global_glance.get_merge_history()
action_full_history = global_glance.get_action_full_history()

/Users/lukasz/miniconda3/envs/globe-ce/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lukasz/Genwro/DiCE/dice_ml/model_interfaces/pytorch_model.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


100%|██████████| 1975/1975 [04:08<00:00,  7.96it/s]


AttributeError: 'NoneType' object has no attribute 'to_numpy'